#Задача №2 - решение задачи регрессии с помощью классической модели регрессии из библиотеки sklearn

Загрузим файл с данными с гугл диска

In [ ]:
! gdown --id 1kD8_4C7Pwe_VUj8zNmUFmKeWGClussAr

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kD8_4C7Pwe_VUj8zNmUFmKeWGClussAr
To: /content/train_sibur_2task.parquet
100% 33.9M/33.9M [00:00<00:00, 55.7MB/s]


Импортируем необходимые библиотеки:

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import pickle
import numpy as np

Откроем файл с данными и посмотрим в каком виде хранятся данные

In [ ]:
data_train = pd.read_parquet("/content/train_sibur_2task.parquet", engine='pyarrow')
data_train.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1
0,32.910908,17.376350,77.557840,2.929855,gas1,20.487150,57.633085,49.245392,-44.124381,23.400064,...,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743
1,41.263782,22.419445,47.945514,-25.847472,gas2,21.461239,3.474080,49.659980,-13.553188,-13.047593,...,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716
2,25.580283,17.376350,77.654180,2.799411,gas1,20.487150,28.938295,49.245392,-44.124381,20.610679,...,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043
3,33.756900,17.376350,73.049625,2.953982,gas1,20.487150,28.932311,49.245392,-44.124381,18.107963,...,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795
4,4.223732,38.772534,48.015553,-25.843943,gas2,24.635721,12.011581,51.030938,84.244199,-17.735680,...,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082


Признак feature4 - это категориальная переменная, которая принимает 2 значения: "gas1" и "gas2". Заменим значения на числовые (0, 1) с помощью LabelEncoder() из библиотеки sklearn

In [ ]:
le = preprocessing.LabelEncoder()
data_train['feature4'] = le.fit_transform(data_train['feature4'])
data_train.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1
0,32.910908,17.376350,77.557840,2.929855,0,20.487150,57.633085,49.245392,-44.124381,23.400064,...,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743
1,41.263782,22.419445,47.945514,-25.847472,1,21.461239,3.474080,49.659980,-13.553188,-13.047593,...,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716
2,25.580283,17.376350,77.654180,2.799411,0,20.487150,28.938295,49.245392,-44.124381,20.610679,...,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043
3,33.756900,17.376350,73.049625,2.953982,0,20.487150,28.932311,49.245392,-44.124381,18.107963,...,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795
4,4.223732,38.772534,48.015553,-25.843943,1,24.635721,12.011581,51.030938,84.244199,-17.735680,...,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082


Проверим есть ли в датасете пропущенные значения и дубликаты

In [ ]:
data_train.isna().sum()

feature0     0
feature1     0
feature2     0
feature3     0
feature4     0
feature5     0
feature6     0
feature7     0
feature8     0
feature9     0
feature10    0
feature11    0
feature12    0
feature13    0
feature14    0
feature15    0
feature16    0
feature17    0
feature18    0
feature19    0
feature20    0
feature21    0
feature22    0
feature23    0
feature24    0
target0      0
target1      0
dtype: int64

In [ ]:
data_train.duplicated().sum()

0

Пропусков и дубликатов нет. Приступим непосредственно к обучению. Нам необходимо предсказать параметры 'target0', 'target1'. Разделим данные на тренировочную и валидационную выборку

In [ ]:
X = data_train.drop(['target0','target1'], axis=1)
y0 = data_train['target0']
y1 = data_train['target1']

X_train, X_val, y0_train, y0_val = train_test_split(X, y0,
                                                    test_size=0.25, random_state=42)
X_train, X_val, y1_train, y1_val = train_test_split(X, y1,
                                                    test_size=0.25, random_state=42)

Построим регерессионные модели для 'target0' и 'target1'.

In [ ]:
model_0 = LinearRegression()
model_0.fit(X_train, y0_train)
y0_train_pred = model_0.predict(X_train)
y0_val_pred = model_0.predict(X_val)

model_1 = LinearRegression()
model_1.fit(X_train, y1_train)
y1_train_pred = model_1.predict(X_train)
y1_val_pred = model_1.predict(X_val)

Посмотрим на метрики обученных моделей

In [ ]:
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_absolute_percentage_error(y0_train, y0_train_pred),
        mean_absolute_percentage_error(y0_val, y0_val_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y0_train, y0_train_pred),
        r2_score(y0_val, y0_val_pred)))
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_absolute_percentage_error(y1_train, y1_train_pred),
        mean_absolute_percentage_error(y1_val, y1_val_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y1_train, y1_train_pred),
        r2_score(y1_val, y1_val_pred)))

MSE train: 0.032, test: 0.032
R^2 train: 0.997, test: 0.997
MSE train: 0.402, test: 0.400
R^2 train: 0.971, test: 0.971


В обоих случаях, несмотря на высокий r2_score, средняя абсолютная процентная погрешность довольно высокая. Попробуем данные с фичами (X) предварительно нормализовать и посмотрим как изменяться метрики

In [ ]:
X_normalized = (X - X.mean())/X.std()

In [ ]:
X_train_n, X_val_n, y0_train, y0_val = train_test_split(X_normalized, y0,
                                                    test_size=0.25, random_state=42)
X_train_n, X_val_n, y1_train, y1_val = train_test_split(X_normalized, y1,
                                                    test_size=0.25, random_state=42)

In [ ]:
model_0 = LinearRegression()
model_0.fit(X_train_n, y0_train)
y0_train_pred = model_0.predict(X_train_n)
y0_val_pred = model_0.predict(X_val_n)

model_1 = LinearRegression()
model_1.fit(X_train_n, y1_train)
y1_train_pred = model_1.predict(X_train_n)
y1_val_pred = model_1.predict(X_val_n)
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_absolute_percentage_error(y0_train, y0_train_pred),
        mean_absolute_percentage_error(y0_val, y0_val_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y0_train, y0_train_pred),
        r2_score(y0_val, y0_val_pred)))
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_absolute_percentage_error(y1_train, y1_train_pred),
        mean_absolute_percentage_error(y1_val, y1_val_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y1_train, y1_train_pred),
        r2_score(y1_val, y1_val_pred)))

MSE train: 0.032, test: 0.032
R^2 train: 0.997, test: 0.997
MSE train: 0.402, test: 0.400
R^2 train: 0.971, test: 0.971


Нормализация никак не повлияла на результат

Возможно зависимость таргетов от признаков нелинейная. Попробуем апроксимировать данные полиномом второй степени. Для этого преобразуем данные с помощью PolynomialFeatures из библиотеки sklearn.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(X)

In [ ]:
X_train, X_val, y0_train, y0_val = train_test_split(poly_features, y0,
                                                    test_size=0.25, random_state=42)
X_train, X_val, y0_train, y1_val = train_test_split(poly_features, y1,
                                                    test_size=0.25, random_state=42)

посмотрим как изменяться метрики

In [ ]:
model_0 = LinearRegression()
model_0.fit(X_train, y0_train)
y0_train_pred = model_0.predict(X_train)
y0_val_pred = model_0.predict(X_val)

model_1 = LinearRegression()
model_1.fit(X_train, y1_train)
y1_train_pred = model_1.predict(X_train)
y1_val_pred = model_1.predict(X_val)
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_absolute_percentage_error(y0_train, y0_train_pred),
        mean_absolute_percentage_error(y0_val, y0_val_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y0_train, y0_train_pred),
        r2_score(y0_val, y0_val_pred)))
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_absolute_percentage_error(y1_train, y1_train_pred),
        mean_absolute_percentage_error(y1_val, y1_val_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y1_train, y1_train_pred),
        r2_score(y1_val, y1_val_pred)))

MSE train: 0.053, test: 0.637
R^2 train: 0.999, test: 0.113
MSE train: 0.053, test: 0.052
R^2 train: 0.999, test: 0.999


Показатели по 'target0' ухудшили, в то время как по 'target1' существенно улучшились, что говорит о том, что 'target1' нелинейно зависит от признаков. 

Сохраним полученные модели в файл

In [ ]:
target0 = "target0.pkl"
with open(target0, 'wb') as file:
    pickle.dump(model_0, file)
target1 = "target1.pkl"
with open(target1, 'wb') as file:
    pickle.dump(model_1, file)

Подготовим сабмит и запишем файл предикт

In [ ]:
%%writefile predict.py
import pandas as pd
from sklearn import preprocessing
import pickle
import pathlib


le = preprocessing.LabelEncoder()



DATA_DIR = pathlib.Path(".")
MODEL_FILE_0 = pathlib.Path(__file__).parent.joinpath("target0.pkl")
MODEL_FILE_1 = pathlib.Path(__file__).parent.joinpath("target1.pkl")

with open(MODEL_FILE_0, 'rb') as f:
    MODEL_0 = pickle.load(f)
with open(MODEL_FILE_1, 'rb') as f:
    MODEL_1 = pickle.load(f)


def predict(df: pd.DataFrame) -> pd.DataFrame:
    """
    Вычисление предсказаний.

    Параметры:
        df:
          датафрейм, содержащий строки из тестового множества.
          Типы и имена колонок совпадают с типами и именами в ноутбуке, не содержит `np.nan` или `np.inf`.

    Результат:
        Датафрейм предсказаний.
        Должен содержать то же количество строк и в том же порядке, а также колонки `target0` и `target1`.
    """

    predictions = {}

    df['feature4'] = le.fit_transform(df['feature4'])
    FTS = df.columns

    for target, model in zip(["target0", "target1"], [MODEL_0, MODEL_1]):

        predictions[target] = model.predict(df[FTS])

    preds_df = pd.DataFrame(predictions, index=df.index)
    return preds_df

Writing predict.py


Запишем файл requirements.txt с версиями используемых библиотек

In [ ]:
!pip3 freeze > requirements.txt